<a href="https://colab.research.google.com/github/StefanHubner/MachineLearningEconomics/blob/main/Autograd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This takes about 9-10 minutes

Sys.setenv("CUDA" = "cpu")
Sys.setenv("TORCH_INSTALL" = "1")
install.packages("torch", reinstall = FALSE)
torch::install_torch()

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘Rcpp’, ‘coro’




In [5]:
install.packages("ISLR")
install.packages("IRdisplay")

library(IRdisplay)
library(ISLR)

data("Default")
Default$default1 <- as.double(Default$default == "Yes")

n <- nrow(Default)
traini <- sample(1:n, n/2)
testi <- -traini

train <- Default[traini,]

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [6]:
library(torch)

x1x2 <- cbind(1, as.matrix(train[,c("balance", "income")])/1000)
y1 <- train$default1

Sys.setenv("CUDA" = "cuda")
# by default it will use the CPU, cuda refers to the GPU
dev <- ifelse(cuda_is_available(), "cuda", "cpu")
cat(paste0("Running on: ", dev))

Running on: cpu

In [27]:
y <- torch_tensor(y1, dtype = torch_float64(), requires_grad = TRUE)$view(c(-1, 1))
x <- torch_tensor(x1x2, dtype = torch_float64(), requires_grad = TRUE)
beta <- torch_randn(3, 1, dtype = torch_float64(), requires_grad = TRUE)
beta <- torch_zeros(3, 1, dtype = torch_float64(), requires_grad = TRUE)


y$shape
x$shape
beta$shape

[1] 5000    1

[1] 5000    3

[1] 3 1

In [42]:
beta <- torch_zeros(3, 1, dtype = torch_float64(), requires_grad = TRUE)

# Define loss function: likelihood (sometimes called binary cross-entropy in ML context)
ll <- function(beta) {
  xbhat <- torch_mm(x, beta)
  yhat <- torch_sigmoid(xbhat)
  loss <- -torch_sum(y * torch_log(yhat) + (1 - y) * torch_log(1 - yhat))
  loss
}

optimizer <- optim_lbfgs(beta, lr = .08)

# A 'closure', i.e. the code that is run in each step of the optimiser
calc_loss <- function() {
  optimizer$zero_grad()
  value <- ll(beta)
  value$backward()
  #nn_utils_clip_grad_value_(beta, 20)
  value
}

In [43]:
for (i in 1:100) {
  logl <- optimizer$step(calc_loss) # 20 internal iterations per step
  display_html(sprintf("<tt>%d: %f</tt>", i, as.numeric(logl$detach())))
}

1: 3465.735903

2: 906.784784

3: 897.677945

4: 888.730315

5: 792.461351

...

96: 384.702266

97: 384.702266

98: 384.702266

99: 384.702266

100: 384.702266

In [44]:
as.numeric(beta$detach())

[1] -11.93987722   5.86175377   0.02303832